In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
from os.path import exists as DoesPathExist
train_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv"
train_scores_path = "/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv"
test_logs_path = "/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv"
sample_submission_path = "/kaggle/input/linking-writing-processes-to-writing-quality/sample_submission.csv"

if DoesPathExist(train_logs_path):
    train_logs = pd.read_csv(train_logs_path)
else:
    train_logs = pd.read_csv("./data/train_logs.csv")

if DoesPathExist(train_scores_path):
    train_scores = pd.read_csv(train_scores_path)
else:
    train_scores = pd.read_csv("./data/train_scores.csv")

if DoesPathExist(test_logs_path):
    test_logs = pd.read_csv(test_logs_path)
else:
    test_logs = pd.read_csv("./data/test_logs.csv")
    
if DoesPathExist(sample_submission_path):
    sample_submission = pd.read_csv(sample_submission_path)
else:
    sample_submission = pd.read_csv("./data/sample_submission.csv")
    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('expand_frame_repr', False)

In [ ]:
train_logs.head()

In [ ]:
train_scores.head()

In [ ]:
train_logs.shape

cols_to_drop = ["down_event", "up_event"]
for col in cols_to_drop:
    train_logs = train_logs.drop(col, axis=1) 
train_logs.shape

## Data Analysis

In [ ]:
train_groups: pd.DataFrame.groupby = train_logs.groupby(["id"])
input_counts = train_groups.activity.count()
print(input_counts)

In [ ]:
def FeatureEngineering(df: pd.DataFrame):
    """Computes the feature engineering for training.

    Args:
        df (pd.DataFrame): Input dataframe.

    Returns:
        pd.DataFrame: This will be prepped with everything needed to be inputed as x-values for a dataframe.
    """
    groups = df.groupby("id")
    tails = groups.tail(1)
    valueCounts = groups["activity"].value_counts()

    result = pd.DataFrame(
        {
            "id": groups["id"],
            "num_events": tails["event_id"],
            "word_count": tails["word_count"],
           # "insert_count": insert_count,
           # "Remove/Cut" : removal_count
        }
    )


    return result

train_engineered = FeatureEngineering(df=train_logs)
train_engineered.info()

In [ ]:
def NormalizeTrainData(input: pd.DataFrame) -> pd.DataFrame:
    min_max_scaler = preprocessing.MinMaxScaler()
    input["num_events"] = min_max_scaler.fit_transform(input[["num_events"]])
    input["word_count"] = min_max_scaler.fit_transform(input[["word_count"]])
    return input

train_data = NormalizeTrainData(train_engineered)

In [ ]:
def OneHotEncoding(n: float):
    result = [0]*12
    result[int(n / 0.5) - 1] = 1
    return result

x_train = train_data.sort_values("id").drop("id", axis=1)
raw_scores = train_scores.sort_values("id").drop("id", axis=1)
y_train = np.array([OneHotEncoding(float(y)) for y in raw_scores['score']])

In [ ]:
num_train = len(y_train)
x_valid = x_train[:int(num_train*0.2)]
y_valid = y_train[:int(num_train*0.2)]
x_train = x_train[int(num_train*0.2):]
y_train = y_train[int(num_train*0.2):]

In [ ]:
model = Sequential([
    Dense(units=512, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=512, activation='relu'),
    Dense(units=12, activation="sigmoid")
])

In [ ]:
predictions = model(np.array(x_train)).numpy()
predictions
tf.nn.softmax(predictions).numpy()
loss_fn = tf.keras.losses.CategoricalCrossentropy()
loss_fn(y_train, predictions).numpy()
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, verbose=2)

In [ ]:
results = model.evaluate(x_valid, y_valid)
for a in range(len(results)):
    print(f"{model.metrics_names[a]}: {results[a]:.3f}")

predictions: np.array = model.predict(x_valid)
scaled_Pred = []
for p in predictions:
    max = p.max()
    scaled_Pred.append(np.array([1 if x == max else 0 for x in p]))


In [ ]:
# Generate the submission
test_data = FeatureEngineering(test_logs)
test_data = NormalizeTrainData(test_data)
predictions = []
for index, row in test_data.iterrows():
    x_data = [x for x in row[1:]]
    pred = model.predict([x_data])
    max = pred[0].max()
    pred = [1 if x == max else 0 for x in pred[0]]
    result = 0.5
    for index, p in enumerate(pred):
        if p == 1:
            result = 0.5 * index + 0.5
    
    predictions.append((row[0], result)) # (Id, result)
print(predictions)

In [ ]:
ids = [p[0] for p in predictions]
scores = [p[1] for p in predictions]
# ids = [1, 2, 3]
# scores = [0.5, 3.0, 4.5]
sample_submission["id"] = ids
sample_submission["score"] = scores
sample_submission[["id", "score"]].to_csv("submission.csv", index=False)